# Using Langchain to create the Q and A pairs from downloaded text Blogs

In [6]:
! pip install python-dotenv

In [7]:
! pip install --upgrade --quiet langchain langchain-core langchain-community langchain-openai

In [2]:
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Get the OpenAI API key from the environment variables
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [55]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-4-turbo") # try with 3.5-turbo since the idea is low cost. However gpt4 geenrates 6 questions so preferred. 
#openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo")

# Basics (Convert One)

In [12]:
HUMAN_TEMPLATE = """
Generate a list of question and answer pairs based on the following context provided. Each Q&A pair should summarize and extract key information relevant to parents of infants, children, and teens. 
    Questions should be direct and address common concerns or topics in pediatric dentistry. 
    Answers must be concise, accurate, and no longer than a few sentences, providing a brief summary of the essential points from the text related to the question.

    Example:
    - Q: What is the recommended age for a child's first dental visit?
    - A: The American Academy of Pediatric Dentistry recommends that a child's first dental visit should be by age 1 or within six months after the first tooth erupts.
    
    Context:
    {context}

    """

In [13]:
CONTEXT = """
Baby's First Laugh

Waiting for your baby's first milestone laugh can be both exciting and frustrating. If you haven't heard it yet, here's how you can get your baby to giggle or laugh for the first time –and you'll remember it forever.

When Should I Expect My Baby to Smile and Laugh for the First Time?
While your baby will make noises and some facial interactions from birth, a social smile will begin to develop around three months of age. By four months, your baby will start smiling spontaneously at people. Once your baby masters smiling and can recognize the positive reactions, sound effects like cooing will begin. Cooing will turn into giggles, and soon after, you will likely hear your baby's first laugh.

How Can I Get My Baby to Smile and Laugh?
Babies won't laugh until they are ready, so while you can and should encourage laughter, don't be discouraged if it isn't happening as soon as you'd hoped.
Try the following to get that first giggle or laugh:
• Copy your baby's sounds
• Act excited and smile when your baby smiles or makes sounds
• Pay close attention to what your baby likes so you can repeat it
• Play games such a peek-a-boo
• Give age-appropriate toys to your baby, such as rattles and picture books
• Put toys near your baby, so they can reach for them or kick them

What if My Baby Smiles a Lot but Doesn't Laugh?
If your baby smiles spontaneously but doesn't seem to want to laugh, you might worry that you're doing something wrong. Don't forget that every baby is born with different innate temperaments, which could influence how much your baby wants to laugh.
That said, the CDC cautions that if your baby hasn't laughed or doesn't laugh regularly by 
age six months old
, you should talk to your baby's doctor or nurse to ensure that this isn't a sign of a possible developmental delay or hearing impairment.
The first few months of your child's life is an exciting time, and 
each milestone
 brings up a whole range of emotions for you as a parent, from your baby's first words to the 
eruption of their first teeth
. Waiting for your baby's first laugh can be frustrating, but remember, the wait is worth it!
"""

In [18]:
from langchain.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("human", HUMAN_TEMPLATE)
])

chat_chain = chat_prompt | openai_chat_model

response=chat_chain.invoke({"context" : CONTEXT})

In [19]:
print(response) # gpt3.5-turbo - creates 3 questions

content="- Q: When should I expect my baby to smile and laugh for the first time?\n- A: A social smile will develop around three months of age, with spontaneous smiling at people starting around four months. Laughter typically follows soon after.\n  \n- Q: How can I encourage my baby to smile and laugh?\n- A: You can encourage laughter by copying your baby's sounds, acting excited when they smile, playing games like peek-a-boo, and giving them age-appropriate toys to interact with.\n\n- Q: What if my baby smiles a lot but doesn't laugh?\n- A: If your baby smiles but doesn't laugh regularly by six months old, it's recommended to talk to your baby's doctor to rule out any possible developmental delays or hearing impairments. \n\n- Q: What are some milestones in a child's first few months of life?\n- A: Milestones in a child's first few months include their first words, the eruption of their first teeth, and their first genuine laughter, among others." response_metadata={'token_usage': {'

In [15]:
print(response) # gpt-4-turbo - More tokens generated. - creates 6 questions. This IMO justifies the additional cost. 
# completion token total is about 35% more for gpt 4 turbo. This works great to generate more questions from same content. 

content="- Q: At what age should I expect my baby to start smiling socially?\n- A: You can expect your baby to begin developing a social smile around three months of age.\n\n- Q: When is it typical for a baby to start laughing?\n- A: Typically, a baby may start laughing around four months old, after they have begun to smile spontaneously at people.\n\n- Q: What are some effective ways to encourage my baby to smile and laugh?\n- A: To encourage your baby to smile and laugh, you can copy their sounds, show excitement when they smile, observe and repeat actions they enjoy, play interactive games like peek-a-boo, and provide age-appropriate toys like rattles and picture books.\n\n- Q: What should I do if my baby smiles but doesn't laugh by six months?\n- A: If your baby hasn't laughed by six months and doesn't laugh regularly, it's advisable to consult your baby's doctor or nurse. This could be important to rule out any developmental delays or hearing impairments.\n\n- Q: Is it a concern i

In [9]:
response.content

"- Q: When should I expect my baby to smile and laugh for the first time?\n- A: A social smile typically develops around three months of age, with spontaneous smiling at people starting around four months. Giggles and the first laugh usually follow soon after.\n\n- Q: How can I encourage my baby to smile and laugh?\n- A: Encourage laughter by copying your baby's sounds, acting excited when they smile, paying attention to their preferences, playing games like peek-a-boo, and providing age-appropriate toys.\n\n- Q: What if my baby smiles a lot but doesn't laugh?\n- A: If your baby smiles but doesn't laugh by age six months, consult your baby's doctor to rule out any developmental delays or hearing impairments. Every baby develops at their own pace. \n\n- Q: What milestones can I expect in the first few months of my child's life?\n- A: The first few months bring various milestones, including the eruption of the first teeth, first words, and other developmental milestones like smiling, coo

In [30]:
# Write to .txt file
with open('faq_training.txt', 'w') as f:
    f.write(response.content)

In [16]:
# Append to .txt file
with open('faq_training.txt', 'a') as f:
    f.write('\n\n'+ response.content)  # format because the next line would like to have a new line

# Create a function that will read directly from .txt and generate Q&A

In [21]:
HUMAN_TEMPLATE = """
Generate a list of question and answer pairs based on the following context provided. Each Q&A pair should summarize and extract key information relevant to parents of infants, children, and teens. 
    Questions should be direct and address common concerns or topics in pediatric dentistry. 
    Answers must be concise, accurate, and no longer than a few sentences, providing a brief summary of the essential points from the text related to the question.

    Example:
    - Q: What is the recommended age for a child's first dental visit?
    - A: The American Academy of Pediatric Dentistry recommends that a child's first dental visit should be by age 1 or within six months after the first tooth erupts.
    
    Context:
    {context}

    """

CONTEXT_FILE_PATH = "/Users/acrobat/Documents/GitHub/extract_html/blogs/baby's_first_laugh.txt"


In [22]:
from langchain.prompts import ChatPromptTemplate

def generate_response(human_template, context_file_path):
    # Read the context from the text file
    with open(context_file_path, 'r') as f:
        context = f.read()

    # Create the chat prompt
    chat_prompt = ChatPromptTemplate.from_messages([
        ("human", human_template)
    ])

    # Create the chat chain
    chat_chain = chat_prompt | openai_chat_model

    # Invoke the chat chain with the context
    response = chat_chain.invoke({"context" : context})

    return response

# Usage

response = generate_response(HUMAN_TEMPLATE, CONTEXT_FILE_PATH)

In [23]:
print(response)

content="- Q: When can I expect my baby to start smiling and laughing?\n- A: A social smile will develop around three months of age, with spontaneous smiling at people starting around four months. Laughter typically follows once your baby masters smiling.\n\n- Q: How can I encourage my baby to smile and laugh?\n- A: You can encourage laughter by copying your baby's sounds, acting excited when they smile or make sounds, playing games like peek-a-boo, and providing age-appropriate toys for them to interact with.\n\n- Q: What should I do if my baby smiles but doesn't laugh?\n- A: If your baby smiles but doesn't laugh regularly by age six months, consult your baby's doctor to rule out any possible developmental delays or hearing impairments. Every baby has a unique temperament that may affect their laughter." response_metadata={'token_usage': {'completion_tokens': 168, 'prompt_tokens': 631, 'total_tokens': 799}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_

# Read each text file one by one and generate questions in the same manner & print to .txt
### Full run by folder - why folder because I may want to add a new folder and append new content to my Q&A file. 

In [75]:
HUMAN_TEMPLATE = """
Generate a list of question and answer pairs based on the following context provided. Each Q&A pair should summarize and extract key information relevant to parents of infants, children, and teens. 
    Questions should be direct and address common concerns or topics in pediatric dentistry. 
    Answers must be concise, accurate, and no longer than a few sentences, providing a brief summary of the essential points from the text related to the question.

    Example:
    Question: What is the recommended age for a child's first dental visit?
    Answer: The American Academy of Pediatric Dentistry recommends that a child's first dental visit should be by age 1 or within six months after the first tooth erupts.
    
    Question: How does a baby's temperament affect their laughter?
    Answer 4: Each baby's innate temperament can influence how frequently they laugh, as some may naturally smile more while others are less inclined to laugh.
    Context:
    {context}

    """

DIRECTORY_PATH = "/Users/acrobat/Documents/GitHub/extract_html/blogs/"


In [76]:
import os
from langchain.prompts import ChatPromptTemplate

def generate_responses(human_template, directory_path):
    # Get the names of all files in the directory
    file_names = os.listdir(directory_path)

    # Loop over the files
    for file_name in file_names:
        # Skip non-text files
        if not file_name.endswith('.txt'):
            continue

        # Construct the full file path
        file_path = os.path.join(directory_path, file_name)

        # Read the context from the text file
        with open(file_path, 'r') as f:
            context = f.read()

        # Create the chat prompt
        chat_prompt = ChatPromptTemplate.from_messages([
            ("human", human_template)
        ])

        # Create the chat chain
        chat_chain = chat_prompt | openai_chat_model

        # Invoke the chat chain with the context
        response = chat_chain.invoke({"context" : context})

        # Print or otherwise use the response here
        #print(response.content)

        #write to file
        with open('faq_teen_13_18.txt', 'a') as f:
            f.write('\n\n'+ response.content) 


In [77]:
generate_responses(HUMAN_TEMPLATE, DIRECTORY_PATH) 
# For a moment I thought the toekn limit may be an issue but since we are making a call each time for each file, no issues.
# For a total of 10 files, it took about 37.1 secs to generate the questions using GPT3.5-turbo.

## Summary: 
#### This Jupyter notebook takes in the .txt files one by one from extracted blogs and then using langchain creates Q&A pairs and writes them to faq_trianing.txt file. Ideally this notebook can be converted to a .py script that works with extract_web_list.py. The reason i chose to seperate this portion as a jupyter notebook is the format of extracted webpages. Extract_web_list will run on Colgate Blogs and other blogs such as Poppy will be manually added to the blogs folder. Future work can be to create a Webflow CMS extract to get Poppy blogs automaticaly but this is not in scope for this project. 


# Next is to convert the faq_training.txt file to a format that can be used to fine tune Mistral 7B Instruct. 

#### If I am using Voiceflow knowledge base then I will have to categorize each Q&A. I will feed the list of questions to OPENAI and then ask it to categorize all questions under a title. This will create a more refined Q and A. this is important because I dont want to look for questions in multiple chuncks. I want similar questions to be stored within same chuncks. 

In [ ]:
#The generation of the data set cost ~$3.49 for numerous blog posts

### Count the number of Question And Answer pairs

In [85]:
def count_qa_pairs(qa_list):
    return len(qa_list)

def read_qa_pairs(file_path):
    with open(file_path, 'r') as f:
        return f.readlines()

file_path = '/Users/acrobat/Documents/GitHub/extract_html/faq_baby_0_4.txt'  
qa_list_from_file = read_qa_pairs(file_path)

num_pairs = count_qa_pairs(qa_list_from_file)
print("Number of question-answer pairs:", num_pairs)

Number of question-answer pairs: 206


In [86]:
file_path = '/Users/acrobat/Documents/GitHub/extract_html/faq_kid_5_12.txt'  
qa_list_from_file = read_qa_pairs(file_path)

num_pairs = count_qa_pairs(qa_list_from_file)
print("Number of question-answer pairs:", num_pairs)

Number of question-answer pairs: 2180


In [87]:
file_path = '/Users/acrobat/Documents/GitHub/extract_html/faq_teen_13_18.txt'  
qa_list_from_file = read_qa_pairs(file_path)

num_pairs = count_qa_pairs(qa_list_from_file)
print("Number of question-answer pairs:", num_pairs)

Number of question-answer pairs: 155


## The number of Q&A Pairs are 2544. Next is merging the data sets and topic modelling. Also for voiceflow I need txt files within each category. But for Mistral finetune I need a json data set. Tpic will be a part of teh data set. I need to somehow divide these QA's into individual topics. 

In [89]:
#first, lets create a single dataset from the 3 files.
# List of your file paths
file_paths = ['/Users/acrobat/Documents/GitHub/extract_html/faq_baby_0_4.txt', '/Users/acrobat/Documents/GitHub/extract_html/faq_kid_5_12.txt', '/Users/acrobat/Documents/GitHub/extract_html/faq_teen_13_18.txt']

# Read the files and store their contents
contents = []
for file_path in file_paths:
    with open(file_path, 'r') as f:
        contents.append(f.read())

# Write the combined contents to a new file
with open('clean_faq_dataset.txt', 'w') as f:
    for content in contents:
        f.write(content + '\n\n')

In [90]:
file_path = '/Users/acrobat/Documents/GitHub/extract_html/clean_faq_dataset.txt'  
qa_list_from_file = read_qa_pairs(file_path)

num_pairs = count_qa_pairs(qa_list_from_file)
print("Number of question-answer pairs:", num_pairs)

Number of question-answer pairs: 2544
